models_plan = """
For TSLA Stock Price Forecasting:
1. ARIMA/SARIMA - Good for capturing trends and seasonality
2. Prophet (Facebook) - Handles missing data and holidays well
3. LSTM (Deep Learning) - Captures complex non-linear patterns
4. Ensemble - Combine multiple models for robustness

For Wealth Projection:
1. Monte Carlo Simulation - Account for uncertainty
2. Scenario Analysis - Bull/Base/Bear cases
3. Regression Models - Model wealth as f(TSLA_price, Other_growth)

Trillionaire Timeline Calculation:
- Project both components separately
- Add them together for total wealth projection
- Calculate when trajectory crosses $1 trillion
- Provide confidence intervals

In [ ]:
# Future price forecasting 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("=" * 70)
print("STEP 6: FORECASTING TSLA FUTURE PRICES")
print("=" * 70)

# Retrain best model on ALL available data for future forecasting
print("\nRetraining best model on complete dataset...")
final_model = auto_arima(
    merged_data['TSLA_Adj_Close'],
    start_p=0, max_p=3,
    start_q=0, max_q=3,
    d=1,
    seasonal=False,
    trace=False,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print(f"Final Model: ARIMA{final_model.order}")

# Forecast future periods
# Let's forecast 5 years (60 months) into the future
forecast_months = 60
print(f"\nForecasting {forecast_months} months into the future...")

# Get forecast with confidence intervals
forecast_result = final_model.predict(n_periods=forecast_months, return_conf_int=True)
forecast_values = forecast_result[0]
confidence_intervals = forecast_result[1]

# Create future date index
last_date = merged_data.index[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), 
                              periods=forecast_months, freq='M')

# Create forecast DataFrame
forecast_df = pd.DataFrame({
    'Date': future_dates,
    'Forecast_Price': forecast_values,
    'Lower_Bound_95': confidence_intervals[:, 0],
    'Upper_Bound_95': confidence_intervals[:, 1]
})
forecast_df.set_index('Date', inplace=True)

print("\nForecast Summary:")
print(forecast_df.head(12))  # First year
print("...")
print(forecast_df.tail(12))  # Last year

# Display key milestones
print("\n" + "-" * 70)
print("FORECAST MILESTONES")
print("-" * 70)
print(f"Current TSLA Price (Nov 2025): ${merged_data['TSLA_Adj_Close'].iloc[-1]:.2f}")
print(f"Forecasted Price (1 year):     ${forecast_df['Forecast_Price'].iloc[11]:.2f}")
print(f"Forecasted Price (3 years):    ${forecast_df['Forecast_Price'].iloc[35]:.2f}")
print(f"Forecasted Price (5 years):    ${forecast_df['Forecast_Price'].iloc[-1]:.2f}")
print(f"\n95% Confidence Interval (5 years): ${forecast_df['Lower_Bound_95'].iloc[-1]:.2f} - ${forecast_df['Upper_Bound_95'].iloc[-1]:.2f}")

In [ ]:
# Calculate Elon's wealth trajectory

print("\n" + "=" * 70)
print("STEP 8: ELON MUSK WEALTH PROJECTION")
print("=" * 70)

# Current parameters (as of Nov 2025)
current_ownership = merged_data['Ownership_Percentage'].iloc[-1] / 100  # Convert to decimal
current_shares = merged_data['Musk_Shares_Millions'].iloc[-1]  # Millions
current_other_wealth = merged_data['Other_Wealth_Billions'].iloc[-1]  # Billions

# Calculate Other Wealth growth rate (historical average)
other_wealth_growth_rate = merged_data['Other_Wealth_Growth'].mean()

print(f"\nCurrent Parameters:")
print(f"  TSLA Ownership: {current_ownership*100:.2f}%")
print(f"  Shares Owned: {current_shares:.1f} million")
print(f"  Other Wealth (SpaceX, xAI, etc.): ${current_other_wealth:.1f}B")
print(f"  Avg Other Wealth Growth Rate: {other_wealth_growth_rate:.4f} per month")

# Project future wealth
wealth_projection = []

for i, date in enumerate(forecast_df.index):
    # TSLA wealth component
    tsla_price = forecast_df['Forecast_Price'].iloc[i]
    tsla_wealth = (tsla_price * current_shares) / 1000  # Convert to billions
    
    # Other wealth component (grows at historical rate)
    months_forward = i + 1
    other_wealth = current_other_wealth * ((1 + other_wealth_growth_rate) ** months_forward)
    
    # Total wealth
    total_wealth = tsla_wealth + other_wealth
    
    wealth_projection.append({
        'Date': date,
        'TSLA_Price': tsla_price,
        'TSLA_Wealth_B': tsla_wealth,
        'Other_Wealth_B': other_wealth,
        'Total_Wealth_B': total_wealth
    })

wealth_df = pd.DataFrame(wealth_projection)
wealth_df.set_index('Date', inplace=True)

print("\nWealth Projection Summary:")
print(wealth_df.head(12))
print("...")
print(wealth_df.tail(12))

In [ ]:
# Assessing Elon's Trillionaire Timeline

print("\n" + "=" * 70)
print("🎯 WHEN WILL ELON MUSK BECOME A TRILLIONAIRE?")
print("=" * 70)

# Find when he crosses $1 trillion
trillion = 1000  # Billions

trillionaire_rows = wealth_df[wealth_df['Total_Wealth_B'] >= trillion]

if len(trillionaire_rows) > 0:
    trillionaire_date = trillionaire_rows.index[0]
    trillionaire_wealth = trillionaire_rows['Total_Wealth_B'].iloc[0]
    tsla_price_at_trillion = trillionaire_rows['TSLA_Price'].iloc[0]
    
    # Calculate time from now
    months_until = (trillionaire_date.year - merged_data.index[-1].year) * 12 + \
                   (trillionaire_date.month - merged_data.index[-1].month)
    years_until = months_until / 12
    
    print(f"\n🚀 PROJECTION RESULTS:")
    print(f"   Trillionaire Date: {trillionaire_date.strftime('%B %Y')}")
    print(f"   Time from Now: {years_until:.1f} years ({months_until} months)")
    print(f"   Projected Wealth: ${trillionaire_wealth:.1f} Billion")
    print(f"   Required TSLA Price: ${tsla_price_at_trillion:.2f}")
    print(f"\n   Breakdown at $1T:")
    print(f"      TSLA Wealth: ${trillionaire_rows['TSLA_Wealth_B'].iloc[0]:.1f}B")
    print(f"      Other Wealth: ${trillionaire_rows['Other_Wealth_B'].iloc[0]:.1f}B")
else:
    print("\n⚠️ Based on current projections, $1 trillion may not be reached within 5 years.")
    print(f"   Max projected wealth: ${wealth_df['Total_Wealth_B'].max():.1f}B")
    print(f"   At: {wealth_df['Total_Wealth_B'].idxmax().strftime('%B %Y')}")
    
    # Calculate what TSLA price would be needed
    current_other = wealth_df['Other_Wealth_B'].iloc[-1]
    needed_tsla_wealth = trillion - current_other
    needed_tsla_price = (needed_tsla_wealth * 1000) / current_shares
    
    print(f"\n   To reach $1T by {forecast_df.index[-1].strftime('%Y')}:")
    print(f"      Required TSLA Price: ${needed_tsla_price:.2f}")
    print(f"      Current Forecast: ${forecast_df['Forecast_Price'].iloc[-1]:.2f}")

# Current status
current_networth = merged_data['Net_Worth_Billions'].iloc[-1]
remaining = trillion - current_networth

print(f"\n📊 CURRENT STATUS:")
print(f"   Current Net Worth: ${current_networth:.1f}B")
print(f"   To $1 Trillion: ${remaining:.1f}B more needed")
print(f"   Progress: {(current_networth/trillion)*100:.1f}%")




# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('LSTM Model Loss', fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['mae'], label='Training MAE')
axes[1].plot(history.history['val_mae'], label='Validation MAE')
axes[1].set_title('LSTM Model MAE', fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ LSTM training history saved!")
Step 4: Compare All Models
pythonprint("\n" + "=" * 70)
print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 70)

# Compile all metrics
model_comparison = pd.DataFrame({
    'Model': [
        'ARIMA(1,1,1)', 
        'Auto ARIMA(0,1,0)', 
        'Prophet',
        'LSTM'
    ],
    'RMSE': [
        metrics_111['RMSE'],
        metrics_auto['RMSE'],
        prophet_rmse,
        lstm_rmse
    ],
    'MAE': [
        metrics_111['MAE'],
        metrics_auto['MAE'],
        prophet_mae,
        lstm_mae
    ],
    'MAPE': [
        metrics_111['MAPE'],
        metrics_auto['MAPE'],
        prophet_mape,
        lstm_mape
    ],
    'R²': [
        metrics_111['R2'],
        metrics_auto['R2'],
        prophet_r2,
        lstm_r2
    ]
})

# Format for display
display_comparison = model_comparison.copy()
display_comparison['RMSE'] = display_comparison['RMSE'].apply(lambda x: f'${x:,.2f}')
display_comparison['MAE'] = display_comparison['MAE'].apply(lambda x: f'${x:,.2f}')
display_comparison['MAPE'] = display_comparison['MAPE'].apply(lambda x: f'{x:.2f}%')
display_comparison['R²'] = display_comparison['R²'].apply(lambda x: f'{x:.4f}')

print("\n")
print(display_comparison.to_string(index=False))

# Find best model by RMSE
best_idx = model_comparison['RMSE'].idxmin()
best_model_name = model_comparison.loc[best_idx, 'Model']
best_rmse = model_comparison.loc[best_idx, 'RMSE']

print(f"\n🏆 Best Individual Model: {best_model_name}")
print(f"   RMSE: ${best_rmse:,.2f}")
print(f"   MAPE: {model_comparison.loc[best_idx, 'MAPE']:.2f}%")
Step 5: Visual Comparison of All Models
python# Compare all predictions visually
fig, axes = plt.subplots(2, 1, figsize=(18, 12))

# Plot 1: Full timeline
axes[0].plot(train_data.index, train_data['TSLA_Adj_Close'], 
             label='Training Data', color='#0066cc', linewidth=2, alpha=0.7)
axes[0].plot(test_data.index, test_data['TSLA_Adj_Close'], 
             label='Actual Test Data', color='#00cc00', linewidth=3)
axes[0].plot(test_data.index, forecast_111, 
             label='ARIMA(1,1,1)', color='#cc0000', linewidth=2, linestyle='--', alpha=0.7)
axes[0].plot(test_data.index, forecast_auto, 
             label='Auto ARIMA(0,1,0)', color='#ff6600', linewidth=2, linestyle='--', alpha=0.7)
axes[0].plot(test_data.index, prophet_predictions, 
             label='Prophet', color='#9900cc', linewidth=2, linestyle='--', alpha=0.7)
axes[0].plot(test_data.index[seq_length:], lstm_predictions, 
             label='LSTM', color='#00cccc', linewidth=2, linestyle='--', alpha=0.7)

axes[0].axvline(x=train_data.index[-1], color='black', linestyle=':', 
                linewidth=2, alpha=0.6, label='Train/Test Split')
axes[0].set_title('All Models: Predictions vs Actual TSLA Stock Price', 
                  fontsize=16, fontweight='bold')
axes[0].set_ylabel('TSLA Price ($)', fontsize=12)
axes[0].legend(loc='upper left', fontsize=10, ncol=2)
axes[0].grid(True, alpha=0.3)

# Plot 2: Test period only (zoomed)
axes[1].plot(test_data.index, test_data['TSLA_Adj_Close'], 
             label='Actual', color='#00cc00', linewidth=3, marker='o', markersize=4)
axes[1].plot(test_data.index, forecast_auto, 
             label=f'ARIMA ({best_rmse:.0f})', color='#ff6600', 
             linewidth=2.5, marker='s', markersize=3, linestyle='--')
axes[1].plot(test_data.index, prophet_predictions, 
             label=f'Prophet ({prophet_rmse:.0f})', color='#9900cc', 
             linewidth=2.5, marker='^', markersize=3, linestyle='--')
axes[1].plot(test_data.index[seq_length:], lstm_predictions, 
             label=f'LSTM ({lstm_rmse:.0f})', color='#00cccc', 
             linewidth=2.5, marker='d', markersize=3, linestyle='--')

axes[1].set_title('Test Period Detail: All Model Predictions', 
                  fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('TSLA Price ($)', fontsize=12)
axes[1].legend(loc='best', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('all_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Comprehensive model comparison plot saved!")